# AIPI 590 - XAI | Assignment 07

Description: In this assignment we will generate a hypothesis regarding a deep learning model and test said hypothesis utilizing saliency mapping for a given image.

John Coogan

Inspired by an example pitfall for machine learning: that models trained to detect planes vs tanks ended up activating on the sky since that was the prime delimeter between the two classes, we will attempt to assess a size bias for ResNet50.

$ H_0$: ResNet50’s classification accuracy and salience map activations are not significantly affected by the size of the object in the image. Small objects in large images are classified with similar accuracy and focus as larger objects, showing no bias in activation towards object size.

$ H_1$: ResNet50 struggles with classifying small objects in large images, indicating that salience maps will show low activation on the object of interest when it occupies a small portion of the image.

In [1]:
!pip install numpy==1.25.2 matplotlib==3.7.1 tensorflow==2.14.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
 

In [2]:
# Basic
import numpy as np
import matplotlib.pyplot as plt

# Model Utils
import tensorflow as tf

# Data Utils
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing import image
from keras.utils import get_file
import json

In [3]:
# Function Author Dr. Brinnae Bent: https://github.com/AIPI-590-XAI/Duke-AI-XAI/blob/main/explainable-ml-example-notebooks/saliency_maps.ipynb
# Function to generate a saliency map for an input image based on a given model


def generate_saliency_map(model, img):
    # Convert the input image to a TensorFlow variable
    x = tf.Variable(img)

    # Add an extra dimension to the image tensor to match the model's input shape
    x = tf.expand_dims(x, axis=0)

    # Preprocess the image according to ResNet50 requirements
    x = tf.keras.applications.resnet50.preprocess_input(x)

    # Create a gradient tape context to record operations for automatic differentiation
    with tf.GradientTape() as tape:
        # Watch the input tensor to calculate gradients
        tape.watch(x)

        # Forward pass: get model predictions for the input image
        preds = model(x)

        # Find the index of the highest predicted class probability
        top_pred_index = tf.argmax(preds[0])

    # Calculate the gradients of the top prediction with respect to the input image
    grads = tape.gradient(preds, x)

    # Compute the saliency map by taking the maximum absolute gradient across color channels
    saliency = tf.reduce_max(tf.abs(grads), axis=-1)[0]

    # Return the saliency map and the index of the top predicted class as a numpy array
    return saliency, top_pred_index.numpy()

In [8]:
index = 100 #Change the index to get a different image

# Load ResNet50 pre-trained model
model = tf.keras.applications.ResNet50(weights='imagenet', include_top=True)

# Load dataset from ImageNetV2
dataset, info = tfds.load('imagenet_v2', with_info=True, split="test[:5%]", as_supervised=True)
for img, label in dataset.take(index):
    img = tf.image.resize(img, (224, 224))
    img = img.numpy().astype(np.float32)

# Download the ImageNet class index file
class_index_path = get_file('imagenet_class_index.json', 'https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json')

# Load ImageNet class labels
with open(class_index_path) as f:
    class_idx = json.load(f)




In [22]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import json
from tensorflow.keras.utils import get_file

class_label = 'cat'

# Load ResNet50 pre-trained model
model = tf.keras.applications.ResNet50(weights='imagenet', include_top=True)

# Load ImageNetV2 dataset
dataset, info = tfds.load('imagenet_v2', with_info=True, split="test[:5%]", as_supervised=True)

# Download the ImageNet class index file
class_index_path = get_file('imagenet_class_index.json', 'https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json')

# Load ImageNet class labels
with open(class_index_path) as f:
    class_idx = json.load(f)

# Find the class index class label
bee_class_index = None
for key, value in class_idx.items():
    if class_label in value[1]:
        bee_class_index = int(key)
        print(f"{class_label} class index found: {bee_class_index}")
        break

# Map TensorFlow dataset labels to ImageNet class labels
class_images = []
count = 0

# The TensorFlow dataset labels might be different from original ImageNet class indices.
# A direct comparison with the ImageNet labels may not work.
for img, label in dataset:
    # Convert TensorFlow label to numpy and adjust comparison
    label = label.numpy()

    # Check if the label corresponds to the  class
    if class_idx[str(label)][1] == class_label:
        print(f"Found {class_label} image at index {count}")
        print(img)
        img = tf.image.resize(img, (224, 224))
        try:
            img_np = img_resized.numpy().astype(np.float32)
            class_images.append(img_np)  # Append only after successful conversion
            count += 1
            print(f"Appended image {count} to class_images.")
        except Exception as e:
            print(f"Error converting image to NumPy: {e}")
        class_images.append(img)
        count += 1

    if count == 3:  # Stop after getting 3 images
        break

# Print out confirmation
print(f"Found {len(class_images)} {class_label} images.")


cat class index found: 143
Found 0 cat images.


In [11]:
# Generate saliency map
saliency_map, top_pred_index = generate_saliency_map(model, bee_images[0])

# Map the index to class label
predicted_class = class_idx[str(top_pred_index)][1]


# Display the original image
plt.figure(figsize=(8, 8))

plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img.astype(np.uint8))
plt.axis('off')
plt.tight_layout()

# Display the saliency map
plt.subplot(1, 2, 2)
plt.imshow(saliency_map, cmap='viridis')
plt.title("Saliency Map")
plt.suptitle(f"Predicted object class: {predicted_class}")
plt.axis('off')
plt.tight_layout()

plt.show()

IndexError: list index out of range